In [53]:
using Rocket
using ReactiveMP
using GraphPPL
using MacroTools

In [26]:
function foo3(args::Vararg{ <: Union{ <: ConstVariable{ <: Dirac }, <: DataVariable{ <: Any, <: Dirac } } })
    return 2
end

foo3 (generic function with 2 methods)

In [60]:
c1 = constvar(:name, 1)
c2 = constvar(:name, 1.0)
d1 = datavar(:n, Dirac{Float64})
d2 = datavar(:n, Dirac{Int})

foo3(c1, c2, d1, d2)

LoadError: UndefVarError: foo3 not defined

In [63]:
MacroTools.prettify(@macroexpand ReactiveMP.@node(
    form       => NormalMeanVariance,
    formtype   => NormalMeanVariance,
    sdtype     => Stochastic,
    interfaces => [
        out,
        (μ, aliases = [ mean ]),
        (v, aliases = [ var ])
    ]
))

quote
    function ReactiveMP.make_node(::Type{<:NormalMeanVariance}; factorisation = ((1, 2, 3),), meta = nothing)
        return FactorNode(NormalMeanVariance, Stochastic, (:out, :μ, :v), collect_factorisation(NormalMeanVariance, factorisation), meta)
    end
    function ReactiveMP.make_node(::Type{<:NormalMeanVariance}, out::AbstractVariable, μ::AbstractVariable, v::AbstractVariable; factorisation = ((1, 2, 3),), meta = nothing)
        node = make_node(NormalMeanVariance, factorisation = factorisation, meta = meta)
        connect!(node, :out, out)
        connect!(node, :μ, μ)
        connect!(node, :v, v)
        return node
    end
    ReactiveMP.interface_get_index(::Type{Val{:NormalMeanVariance}}, ::Type{Val{:out}}) = 1
    ReactiveMP.interface_get_name(::Type{Val{:NormalMeanVariance}}, ::Type{Val{:out}}) = :out
    ReactiveMP.interface_get_name(::Type{Val{:NormalMeanVariance}}, ::Type{Val{1}}) = :out
    ReactiveMP.interface_get_index(::Type{Val{:NormalMeanVariance}}, ::Type

In [29]:
(a = 1, )

(a = 1,)

In [12]:
function __extract_interfaces_macro_rule(interfaces)
    return interfaces
end

__extract_interfaces_macro_rule (generic function with 1 method)

In [16]:
__extract_interfaces_macro_rule(:([
    out,
    (μ, aliases = [ mean ]),
    (v, aliases = [ var ])
]))

:([out, (μ, aliases = [mean]), (v, aliases = [var])])